<p style="align: center;"><img align=center src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" style="height:90px;" width=500/></p>

<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

## Text to speech: семинар

Привет! 

В этом ноутбуке мы позапускаем модели, которые были описаны на лекции: Tacotron, WaveNet, DeepVoice. Обучать сами в этот раз ничего не будем, поскольку нормальные модели для text-to-speech обучаются как минимум сутки (но это не говорит о том, что не нужно пытаться обучать -- просто в рамках занятия не успеем это сделать до нормального качества физически).

<img src="https://s.tcdn.co/163/0f3/1630f39d-45fe-3d77-8d1d-1b2cf25f380d/1.png" width=300>

Напомню, задача стоит так: **по входному тексту (набору символов) получить на выходе аудиофайл, в котором человек тем или иным голосом произносит данный текст**, то есть задача **text-to-speech**.  

Сразу можно заметить, что постановка уже подразумевает несколько трактовок: с какой именно интонацией произнести? каким именно голосом? акцент? паузы? скорость речи?

Всё это определяется уже на этапе генерации речи нашей моделью.

## Небольшое введение

Чтобы лучше разобраться, что вообще подаётся на вход и как работать со звуком в цифровом виде, всем очень рекомендую прочитать [статью от Центра Речевых Технологий (ЦРТ)](https://habr.com/ru/company/speechpro/blog/358816/), это очень хорошая статья про генерацию речи с помощью DCTTS своими руками.

В целом как обзор методов, которые были раньше (конкатенационные и параметрические), и как обзор некоторых современных методов, неплоха [эта статья](https://medium.com/@saxenauts/speech-synthesis-techniques-using-deep-neural-networks-38699e943861).  


Если выделить ключевые вещи, которые надо понимать при обработке звука, то они могут выглядеть так:
1. Понимание того, как устроена запись звука (опционально)
2. Библиотека [librosa](http://librosa.github.io/librosa )
3. Примерное понимание того, что такое преобразование Фурье и почему оно здесь эффективно. Про него можно почитать [здесь](https://habr.com/ru/post/196374/) и [здесь](https://habr.com/ru/post/269991/)

## Нейросети для генерации речи

Мы рассмотрим модели:
1. WaveNet  
2. Tacotron
3. Tacotron 2
4. Deep Voice 3
5. DCTTS (кратко)

Везде далее надо держать в голове, что есть два этапа в end2end TTS (да, звучит противоречиво, ведь это end2end=):
1. Преобразование текста в спектрограммы 
2. Получение по спектрограммам waveform, а потом из них аудиофайла (этой часть занимаются так называемые **vocoder'ы**)

Для 1 используют Tacotron первой версии, для 2 WaveNet первой версии. Также для 2 используют алгоритм Гриффина-Лима.

Сначала было Tacotron 1 + Griffin-Lim, потом придумали Tacotron 2, который совместил в себе Tacotron 1 и WaveNet.

Также есть DeepVoice, который как Tacotron 1. DCTTS тоже как Tacotron 1 (они  используют постобработку спектрограмм в вейвформы, а сами предсказывают эти самые спектрограммы).

Из датасетов важно знать TIMIT, LibriSpeech и [LJ-Speech](https://keithito.com/LJ-Speech-Dataset/).

### WaveNet

<img src="https://i.ytimg.com/vi/GO_2aVqZlQs/hqdefault.jpg" width=300>

[WaveNet](https://deepmind.com/blog/wavenet-generative-model-raw-audio/) была одна из первых архитектур, придуманных для генарции речи из полученных по тексту фонем. Она [была опубикована 19 сентября 2016 года](https://arxiv.org/pdf/1609.03499.pdf) компанией DeepMind (сейчас является частью Google).

Одна из самых популярных реализаций -- реализация на TensorFlow: https://github.com/ibab/tensorflow-wavenet. Однако, как можно заметить в репозитории, эта модель позволяет генрировать речь несвязно, то есть не по тексту, а просто из шума, но там даже есть условие на то, кто именно будет произносить эту речь (Global Conditionng).



Мы всё же хотим генерировать аудио из текста, WaveNet сам по себе для этого не подходит. Передём к моделям, которые позволят это делать.

Также рекомендую прочитать [более современную статью по WaveNet](https://towardsdatascience.com/wavenet-google-assistants-voice-synthesizer-a168e9af13b1), в том числе про Parallel WaveNet.

### Tacotron

<img src="https://memepedia.ru/wp-content/uploads/2018/10/jv5l3v9svse-1-kopiya.jpg" width=300>

[Tacotron](https://arxiv.org/pdf/1703.10135.pdf) был опубликован 16 апреля 2017 года компанией Google. 

Здесь есть https://github.com/keithito/tacotron (1300+ звёзд на гитхбе) и https://github.com/Kyubyong/tacotron (1400+ звёзд), попробуем запустить первый/

In [0]:
!git clone https://github.com/keithito/tacotron tacotron_1
!ls

In [0]:
import os
os.chdir('./tacotron_1')

In [0]:
!pip install -r requirements.txt

In [0]:
!curl http://data.keithito.com/data/speech/tacotron-20180906.tar.gz | tar xzC /tmp

In [0]:
!python3 demo_server.py --checkpoint /tmp/tacotron-20180906/model.ckpt

У себя на компьютере это сработало бы, но с колаба сервер не запустить (по крайней мере, это не просто сделать). Попробуем второй:

In [0]:
os.chdir('../')
!ls

In [0]:
!git clone https://github.com/Kyubyong/tacotron tacotron_2

In [0]:
!pwd
!ls

In [0]:
os.chdir('./tacotron_2/')

In [0]:
!python synthesize.py

Ошибка, потому что мы не указали, из какой папки брать вес предобученной модели. Скачаем их и положим в нужную папку:

In [0]:
!wget https://www.dropbox.com/s/8kxa3xh2vfna3s9/LJ_logdir.zip

In [0]:
!ls

In [0]:
!unzip LJ_logdir.zip

In [0]:
!ls

In [0]:
!mv LJ logdir
!mkdir logdir/01
!mv logdir/* logdir/01
!ls logdir/01

Снова попробуем запустить:

In [0]:
!python synthesize.py

In [0]:
!ls samples/

In [0]:
from google.colab import files

for i in range(1, 21):
    files.download(f'samples/{i}.wav')

Если послушать, то получилось вполне неплохо.

Итак, мы сделали следующее:
1. Написали текст на вход
2. Нормализовали текст, закодировали символы через char2idx
3. Подали на вход Tacotron'у 
4. Получили на выходе из Tacotron'а mel-спектрограммы
5. Преобразовали их в waveform'ы с помощью алгоритма [Гриффина-Лима](https://github.com/Kyubyong/tacotron/blob/379bb7f54c3359ffe97d1f09a773bc6da49eba6f/utils.py#L107)
6. Произвели постобработку, получили .wav-файлы

Попробуем написать  свой текст:

In [0]:
HEADER = 'http://www.cs.columbia.edu/~hgs/audio/harvard.html\n'
MY_TEXTS = [
    '1. Deep Learning School is the best place to study the neural networks.\n',
    '2. AI, BigData, Blockchain, Agile.\n',
    '3. MIPT PSAMI is not about dogs, it stands for Phystech-School of Mathematics and Informatics.\n'
]
with open('harvard_sentences.txt', 'w') as texts_file:
    texts_file.write(HEADER)
    for text in MY_TEXTS:
        texts_file.write(text)

In [0]:
!cat harvard_sentences.txt

In [0]:
!python synthesize.py

In [0]:
from google.colab import files

for i in range(1, 4):  # сейчас только 1.wav, 2.wav и 3.wav перезаписались
    files.download(f'samples/{i}.wav')

### Tacotron 2

<img src="https://memestatic1.fjcdn.com/comments/Your+grandma+is+a+fox+_1b9c90c0bd43b76f2f364a3b424a1260.jpg" width=300>

[Tacotron 2](https://arxiv.org/pdf/1712.05884.pdf) появился 16 февраля 2018 года, разработан компанией Google (совместно с DeepMind).

Поищем реализации. С виду очень хороший репозиторий: https://github.com/Rayhane-mamah/Tacotron-2 

В нём хорошо описано всё, что нужно для загрузки и предобработки датасета, а также обучения и тестирования модели. Мы могли бы его позапускать, но есть **нюанс -- нет предобученных весов**, придётся оубчать с нуля (а мы хотим поскорее запустить) :(



In [0]:
# !git clone https://github.com/Rayhane-mamah/Tacotron-2

In [0]:
# !apt-get install -y libasound-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg libav-tools

In [0]:
# !pip3 install -r Tacotron-2/requirements.txt

In [0]:
# !wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2

**НО**: есть ещё один классный репозиторий https://github.com/r9y9/wavenet_vocoder, там есть ноутбуки на колабе (перейдём туда).

### Deep Voice 3

<img src="http://78.media.tumblr.com/b26d78d8fcfcdc98ede414f079d71215/tumblr_inline_npzymfyFPt1sh39fc_500.jpg" width=300>

[Deep Voice 3](https://arxiv.org/pdf/1710.07654.pdf) был опубикован 20 февраля 2018 года совместно Baidu, OpenAI и Университетом Беркли.

Идеальный репозиторий с колаб-ноутбуком, готовым для запуска: https://github.com/r9y9/deepvoice3_pytorch (в него и перейдём)

### DCTTS

По этой модели написана [статья на Хабре от ЦРТ](https://habr.com/ru/company/speechpro/blog/358816/), плюс есть хороший репозиторий: https://github.com/Kyubyong/dc_tts, на ней останавливаться не будем (по функциональности делает то же, что и Tacotron).

### Ещё

Ещё есть Mozilla TTS: https://github.com/mozilla/TTS и OpenSeq2Seq от NVIDIA: https://nvidia.github.io/OpenSeq2Seq/, безусловно, очень полезные библиотеки, попробуйте потестить их самостоятельно.   

если хочется погрузиться в более классические методы обработки звуки фундаментально, то можно пройти [курс от Стэнфорда](https://web.stanford.edu/class/cs224s/syllabus.html)

А мы движемся к ещё более горячей теме :)

## Генерация музыки

<img src="https://www.nastol.com.ua/pic/201209/1920x1080/nastol.com.ua-31871.jpg" width=400> 

На эту тему будет домашнее задание, подробнее там.

Ссылки, по которым стоит пройтись:

* ШОК ЧИТАТЬ ВСЕМ: https://openai.com/blog/musenet/
* https://salu133445.github.io/musegan/
* https://github.com/tensorflow/magenta/tree/master/magenta/models/melody_rnn
* https://en.wikipedia.org/wiki/ABC_notation
* [Статья про генерацию музыки RNN-кой](https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5)